In [1]:
# imports

import os

from IPython.display import Markdown, display, update_display
from dotenv import load_dotenv
from openai import OpenAI
from util import extract_md_links, fetch_urls_as_map



In [2]:

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [3]:
POETRY_DOCS_SITEMAP_LINK = "https://github.com/python-poetry/poetry/tree/main/docs"

In [4]:
def get_poetry_docs():
    urls = extract_md_links(POETRY_DOCS_SITEMAP_LINK)
    return fetch_urls_as_map(urls)

In [5]:
poetry_docs = get_poetry_docs()

In [6]:
def create_prompt(query):
    system_prompt = """
    You are an assistant that answers questions about Poetry using its official documentation.
    Explain concepts and commands clearly and concisely for Python developers.
    Base your responses only on the provided documentation content.
    If a question is not covered by the provided Poetry documentation, clearly state that it is out of scope and do not provide additional information.
    Respond in Markdown.
    """
    user_prompt = """
    You are looking at the Poetry documentation.
    Here are the contents of several relevant documentation pages.
    Use this information to answer the user’s question clearly and accurately.\n\n
    """
    for _, value in poetry_docs.items():
        user_prompt += f"\n\n{value}\n\n"
    user_prompt += f"\n\nUser Question: {query}\n\n"

    prompt = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
    return prompt

In [7]:
def print_streamed_response(stream_response):
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream_response:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)
def print_full_response(full_response):
    result = full_response.choices[0].message.content
    display(Markdown(result))

In [8]:
def poetry_query_helper(query, stream=False):
    prompt = create_prompt(query)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=prompt,
        stream=stream
    )
    if stream:
        print_streamed_response(response)
    else:
        print_full_response(response)
    return None

In [9]:
poetry_query_helper("How does Redis handle persistence and replication, and what should I enable in production?")



This question is out of scope for the provided Poetry documentation. The materials you shared cover Poetry features (dependencies, packaging, environments, configuration, etc.) but do not discuss Redis persistence or replication, or production deployment specifics for Redis.

If you’d like, I can help answer questions about Poetry-related production considerations (e.g., how to configure environments, dependency locking, CI recommendations) using the documented content.

In [11]:
poetry_query_helper("How does Poetry resolve dependency version conflicts when multiple transitive dependencies specify incompatible constraints, and how is this reflected in poetry.lock?", stream=True)

- Resolution process
  - Poetry uses a dependency resolver to pick exact package versions that satisfy all constraints across the dependency graph. It searches configured repositories for candidate versions and tries to find a combination that satisfies every dependency (including transitive ones).
  - If multiple transitive constraints constrain the same package to incompatible versions, the solver will attempt to find a compatible combination. If no such combination exists, dependency resolution fails and you’ll see an resolution error.
  - When constraints can be satisfied, Poetry selects a set of concrete versions that meet all constraints and proceeds to install them.

- How this is reflected in poetry.lock
  - After a successful resolution, Poetry writes the chosen exact versions to the poetry.lock file. The lock file records the specific versions that were installed (including transitive dependencies) so every environment uses the same set.
  - If you later run poetry install with an existing lock file, Poetry installs the exact versions listed in poetry.lock, ensuring reproducible installs across environments.
  - If you want to explore newer compatible versions, you can run poetry update, which re-solves dependencies and updates the lock file with the latest matching versions according to your pyproject.toml.
  - Poetry also warns when the lock file is out of sync with pyproject.toml, indicating that the lock and metadata should be reconciled (e.g., by running install/update or regenerating the lock).

- Note on multiple constraints
  - Poetry supports multiple constraints for the same package (e.g., via multiple dependency entries or expanded constraint forms). The resolver must find a single set of versions that satisfies all of these constraints; otherwise, resolution fails. The lock will reflect the final resolved versions chosen by the solver.